In [1]:
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.transform import linear_cmap
from bokeh.util.hex import hexbin
from bokeh.plotting import figure
from scipy.signal import savgol_filter

from bokeh.io import curdoc
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, DataRange1d, Select
from bokeh.palettes import Blues4
from bokeh.plotting import figure
import matplotlib
from matplotlib.pyplot import plot as plt

/home/karine/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/karine/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
#READ Files
meteo = pd.read_excel("Weather Data 2014-11-30.xlsx",na_values=['---','+'])
power_networkendpoint = pd.read_csv("network.csv")
feeders = pd.read_csv("feeders.csv")
endpoints = pd.read_csv("Customer_Endpoints.csv")

/home/karine/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
meteo['SolarRad']= pd.to_numeric(meteo['SolarRad'])
SolarRad = meteo['SolarRad'][:]
meteo['SolarRad'].dropna()
SolarRad = meteo['SolarRad'][:len(endpoints['datetime'])*2]

rng2 = pd.date_range('2013-10-01 12:00:00', periods=len(SolarRad ), freq='30min')
df_ = pd.DataFrame({'Period': rng2, 'Solar_Rad':SolarRad })

rng = pd.date_range('2013-10-01 12:00:00', periods=len(endpoints['datetime']), freq='H')

df = pd.DataFrame({'Period': pd.to_datetime(endpoints['datetime']), 'Pmax':endpoints['P_IMPORT_MAX'], 'Pmin':endpoints['P_IMPORT_MIN'] })

In [4]:
df_.head()

,Period,Solar_Rad
0,2013-10-01 12:00:00,32.0
1,2013-10-01 12:30:00,20.0
2,2013-10-01 13:00:00,22.0
3,2013-10-01 13:30:00,8.0
4,2013-10-01 14:00:00,0.0


In [5]:
df.head()

,Period,Pmax,Pmin
0,2013-10-01 12:00:00,-1.734,-1.870
1,2013-10-01 13:00:00,0.896,-1.789
2,2013-10-01 14:00:00,0.175,-1.761
3,2013-10-01 15:00:00,-0.327,-1.337
4,2013-10-01 16:00:00,-0.017,-0.216


In [6]:
Data = pd.merge(df,df_,on="Period")
Data = Data.dropna()
Data.head()

,Period,Pmax,Pmin,Solar_Rad
0,2013-10-01 12:00:00,-1.734,-1.870,32.0
1,2013-10-01 12:00:00,0.113,-0.074,32.0
2,2013-10-01 12:00:00,0.016,-0.924,32.0
4,2013-10-01 13:00:00,0.896,-1.789,22.0
5,2013-10-01 13:00:00,-0.038,-0.233,22.0


In [7]:
import numpy as np

from bokeh.plotting import figure, output_file, show
from bokeh.layouts import gridplot

# prepare some data
aapl = np.array(Data['Pmax'])
solar =  np.array(Data['Solar_Rad'])

aapl_dates = np.array(Data['Period'], dtype=np.datetime64)

window_size = 24
window = np.ones(window_size)/float(window_size)
aapl_avg = np.convolve(aapl, window, 'same')
window_size = 24
window = np.ones(window_size)/float(window_size)
solar_rad_avg = np.convolve(solar, window, 'same')



# output to static HTML file
output_file("P_max.html", title="Pmax.py example")

# create a new plot with a a datetime axis type
s1 = figure(plot_width=800, plot_height=350, x_axis_type="datetime")

# add renderers
s1.circle(aapl_dates, aapl_avg , size=2, color='darkgrey', alpha=0.2, legend='close')

s2 = figure(plot_width=800, plot_height=350, x_axis_type="datetime")

s2.circle(aapl_dates, solar_rad_avg , size=2, color='blue', alpha=0.2, legend='close')


#p.line(aapl_dates, aapl , color='navy', legend='avg')



# show the results
p = gridplot([[s1, s2]])
# NEW: customize by setting attributes
s1.title.text = "Pmax One day average"
s1.legend.location = "top_left"
s1.grid.grid_line_alpha = 0
s1.xaxis.axis_label = 'Date'
s1.yaxis.axis_label = 'PaMax'
s1.ygrid.band_fill_color = "olive"
s1.ygrid.band_fill_alpha = 0.1

s2.title.text = "Solar radiation One day average"
s2.legend.location = "top_left"
s2.grid.grid_line_alpha = 0
s2.xaxis.axis_label = 'Date'
s2.yaxis.axis_label = 'LW'
s2.ygrid.band_fill_color = "blue"
s2.ygrid.band_fill_alpha = 0.1


# show the results
show(p)
aapl
Data.head()
Data.iloc[-1,:]

Period       2014-05-08 10:00:00
Pmax                       0.269
Pmin                       0.269
Solar_Rad                    132
Name: 50448, dtype: object